In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [2]:
data = pd.read_csv('./data/fake_job_postings.csv')
data.head(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [4]:
new_df = data[["title", "location", "company_profile", "description", "requirements", 
               "benefits", "telecommuting","has_company_logo","has_questions","employment_type",
               "required_experience","industry","function","fraudulent"]]
#new_df = new_df.dropna()
#new_df.info()

# 1. Preprocessing

In [5]:
#pip install nltk

In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# text preprocessing function
def preprocess_text(text):

    # tokenization
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package punkt to /Users/lilyw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lilyw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lilyw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/lilyw/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
# replace NA
text_df = new_df[["title", "company_profile", "description", "requirements", "benefits"]]
text_df['title'].fillna('unknown', inplace=True)
text_df['company_profile'].fillna('unknown', inplace=True)
text_df['description'].fillna('unknown', inplace=True)
text_df['requirements'].fillna('unknown', inplace=True)
text_df['benefits'].fillna('unknown', inplace=True)

# process text columns
text_df['preprocessed_title'] = text_df['title'].apply(preprocess_text)
text_df['preprocessed_company_profile'] = text_df['company_profile'].apply(preprocess_text)
text_df['preprocessed_description'] = text_df['description'].apply(preprocess_text)
text_df['preprocessed_requirements'] = text_df['requirements'].apply(preprocess_text)
text_df['preprocessed_benefits'] = text_df['benefits'].apply(preprocess_text)

text_df.head(5)

/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_2800/523841265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['title'].fillna('unknown', inplace=True)
/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_2800/523841265.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['company_profile'].fillna('unknown', inplace=True)
/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_2800/523841265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,title,company_profile,description,requirements,benefits,preprocessed_title,preprocessed_company_profile,preprocessed_description,preprocessed_requirements,preprocessed_benefits
0,Marketing Intern,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,marketing intern,'re food52 've created groundbreaking award-wi...,food52 fast-growing james beard award-winning ...,experience content management system major plu...,unknown
1,Customer Service - Cloud Video Production,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,customer service cloud video production,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get usthrough part 90 second team gain experie...
2,Commissioning Machinery Assistant (CMA),Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,commissioning machinery assistant cma,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre-commissioning commissioning proc...,unknown
3,Account Executive - Washington DC,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,account executive washington dc,passion improving quality life geography heart...,company esri – environmental system research i...,education bachelor ’ master ’ gi business admi...,culture anything corporate—we collaborative cr...
4,Bill Review Manager,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,bill review manager,spotsource solution llc global human capital m...,job title itemization review managerlocation f...,qualification rn license state texasdiploma ba...,full benefit offered


In [8]:
text_df.shape

(17880, 10)

# 2. Text Vectorization

## 2.1 Vectorization method 1: TF-IDF

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

columns_to_vectorize = ["preprocessed_title", "preprocessed_company_profile", "preprocessed_description", "preprocessed_requirements", "preprocessed_benefits"]

tfidf_data = {}

tfidf_vectorizer = TfidfVectorizer(max_features=100)

for column in columns_to_vectorize:

    tfidf_matrix = tfidf_vectorizer.fit_transform(text_df[column])
    tfidf_data[column] = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print(tfidf_data)

{'preprocessed_title':         16   18  abroad   account  admin  administrative  administrator  \
0      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
1      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
2      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
3      0.0  0.0     0.0  0.700986    0.0             0.0            0.0   
4      0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
...    ...  ...     ...       ...    ...             ...            ...   
17875  0.0  0.0     0.0  0.698317    0.0             0.0            0.0   
17876  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17877  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17878  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   
17879  0.0  0.0     0.0  0.000000    0.0             0.0            0.0   

       agent  analyst  android  ...  system  teacher  team  technical  \
0  

In [10]:
# combine dataframe
vect_df = pd.concat([tfidf_data["preprocessed_title"], tfidf_data["preprocessed_company_profile"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_description"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_requirements"]], axis=1)
vect_df = pd.concat([vect_df, tfidf_data["preprocessed_benefits"]], axis=1)
#vect_df = pd.concat([vect_df, new_df["fraudulent"]], axis=1)

y = new_df["fraudulent"]
vect_df.shape

(17880, 500)

## 2.2 Vectorization method 2: BERT

In [13]:
#!pip install transformers

In [26]:
from transformers import BertTokenizer, BertModel
import torch

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
def tokenize_and_encode(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    return tokens

bert_token_data = pd.DataFrame()
bert_token_data['tokenized_title'] = text_df["preprocessed_title"].apply(tokenize_and_encode)
bert_token_data['tokenized_company_profile'] = text_df["preprocessed_company_profile"].apply(tokenize_and_encode)
bert_token_data['tokenized_description'] = text_df["preprocessed_description"].apply(tokenize_and_encode)
bert_token_data['tokenized_requirements'] = text_df["preprocessed_requirements"].apply(tokenize_and_encode)
bert_token_data['tokenized_benefits'] = text_df["preprocessed_benefits"].apply(tokenize_and_encode)

In [28]:
from datetime import datetime

start_time = datetime.now()

def extract_word_embeddings(tokens):
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state

bert_data = pd.DataFrame()
bert_data['word_embedding_title'] = bert_token_data['tokenized_title'].apply(extract_word_embeddings)
elapsed_time = datetime.now() - start_time
print("Elapsed time 1:", elapsed_time.total_seconds(), "seconds")

bert_data['word_embedding_company_profile'] = bert_token_data['tokenized_company_profile'].apply(extract_word_embeddings)
elapsed_time = datetime.now() - start_time
print("Elapsed time 2:", elapsed_time.total_seconds(), "seconds")

bert_data['word_embedding_description'] = bert_token_data['tokenized_description'].apply(extract_word_embeddings)
elapsed_time = datetime.now() - start_time
print("Elapsed time 3:", elapsed_time.total_seconds(), "seconds")

bert_data['word_embedding_requirements'] = bert_token_data['tokenized_requirements'].apply(extract_word_embeddings)
elapsed_time = datetime.now() - start_time
print("Elapsed time 4:", elapsed_time.total_seconds(), "seconds")

bert_data['word_embedding_benefits'] = bert_token_data['tokenized_benefits'].apply(extract_word_embeddings)

end_time = datetime.now()
elapsed_time = end_time - start_time
print("Elapsed time5:", elapsed_time.total_seconds(), "seconds")

bert_data.to_pickle('tensor_bert_data.pkl')
bert_data.to_csv("bert_data.csv")

Elapsed time 1: 515.072433 seconds
Elapsed time 2: 2211.640417 seconds
Elapsed time 3: 5175.414101 seconds
Elapsed time 4: 6943.254818 seconds
Elapsed time5: 7883.984557 seconds


In [16]:
bert_data.head()

,Unnamed: 0,word_embedding_title,word_embedding_company_profile,word_embedding_description,word_embedding_requirements,word_embedding_benefits
0,0,"tensor([[[-0.2762, 0.2952, -0.2674, ..., -0....","tensor([[[-0.1387, 0.0414, 0.3453, ..., -0....","tensor([[[-0.1852, 0.1841, 0.0177, ..., -0....","tensor([[[-0.2802, 0.0395, 0.0526, ..., -0....","tensor([[[-0.4404, 0.1208, -0.3744, ..., -0...."
1,1,"tensor([[[-0.4242, 0.0289, -0.2216, ..., -0....","tensor([[[-0.3376, 0.1549, 0.2261, ..., -0....","tensor([[[-0.0900, 0.1768, 0.6600, ..., -0....","tensor([[[-0.2514, -0.0361, 0.2124, ..., -0....","tensor([[[-0.2629, 0.1672, 0.4715, ..., -0...."
2,2,"tensor([[[-0.4066, 0.3469, -0.2398, ..., -0....","tensor([[[-0.5047, -0.0219, 0.1005, ..., -0....","tensor([[[-0.2912, 0.2402, 0.4106, ..., -0....","tensor([[[-0.2419, -0.0465, 0.6407, ..., -0....","tensor([[[-0.4404, 0.1208, -0.3744, ..., -0...."
3,3,"tensor([[[-0.3563, 0.0420, -0.1740, ..., -0....","tensor([[[-0.4097, -0.3133, 0.1103, ..., -0....","tensor([[[-0.3401, 0.0642, 0.0743, ..., -0....","tensor([[[-0.2363, 0.1603, 0.1603, ..., -0....","tensor([[[ 0.0051, -0.1624, -0.1307, ..., -0...."
4,4,"tensor([[[-0.3451, 0.2884, 0.0088, ..., -0....","tensor([[[-1.8440e-01, 1.9524e-01, 1.8527e-0...","tensor([[[-0.3737, 0.4446, 0.3489, ..., -0....","tensor([[[-0.4553, 0.0736, 0.1529, ..., -0....","tensor([[[-2.5761e-01, 3.7348e-02, -3.8978e-0..."


In [11]:
from transformers import BertTokenizer, BertModel
import torch

bert_data = pd.read_pickle('tensor_bert_data.pkl')
#bert_data = pd.read_csv("bert_data.csv")

/Users/lilyw/opt/anaconda3/envs/ada/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
type(bert_data['word_embedding_title'][0])

torch.Tensor

In [13]:
resized_bert_data = pd.DataFrame()

mean_embeddings = [torch.mean(embedding, dim=0) for embedding in bert_data['word_embedding_title']]
resized_bert_data['word_embedding_title'] = mean_embeddings

In [14]:
mean_embeddings = [torch.mean(embedding, dim=0) for embedding in bert_data['word_embedding_company_profile']]
resized_bert_data['word_embedding_company_profile'] = mean_embeddings

In [15]:
mean_embeddings = [torch.mean(embedding, dim=0) for embedding in bert_data['word_embedding_description']]
resized_bert_data['word_embedding_description'] = mean_embeddings

In [16]:
mean_embeddings = [torch.mean(embedding, dim=0) for embedding in bert_data['word_embedding_requirements']]
resized_bert_data['word_embedding_requirements'] = mean_embeddings

In [17]:
mean_embeddings = [torch.mean(embedding, dim=0) for embedding in bert_data['word_embedding_benefits']]
resized_bert_data['word_embedding_benefits'] = mean_embeddings

In [20]:
resized_bert_data.to_csv("resized_bert_data.csv")

# 3. Model Training

### Split data

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(vect_df, y, test_size=0.2, random_state=42)
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(bert_data, y, test_size=0.2, random_state=42)

In [42]:
# Concatenate BERT embeddings
X_train_bert_concat = np.vstack([np.array(X_train_bert[col].tolist()) for col in X_train_bert.columns])
X_test_bert_concat = np.vstack([np.array(X_test_bert[col].tolist()) for col in X_test_bert.columns])

/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_2800/2682760496.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X_train_bert_concat = np.vstack([np.array(X_train_bert[col].tolist()) for col in X_train_bert.columns])
/var/folders/yt/170z436920j8xfh51ybwr4hr0000gn/T/ipykernel_2800/2682760496.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_be

## 3.1 Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_tfidf = LogisticRegression()
lr_tfidf.fit(X_train_tfidf, y_train_tfidf)

In [56]:
from sklearn.model_selection import cross_val_score

measure_metrics = ['accuracy', 'f1', 'average_precision', 'recall', 'roc_auc']
for sco_name in measure_metrics:
    score = cross_val_score(lr_tfidf, X_train_tfidf, y_train_tfidf, scoring=sco_name, cv=10)
    print("The", sco_name, "score of Logistic Regression model on training set is ", score)

The accuracy score of Logistic Regression model on training set is  [0.97833683 0.9650594  0.96645702 0.97134871 0.97412587 0.96993007
 0.96923077 0.96643357 0.96993007 0.96993007]
The f1 score of Logistic Regression model on training set is  [0.73043478 0.52830189 0.53846154 0.61682243 0.67826087 0.57425743
 0.59259259 0.55555556 0.63865546 0.59047619]
The average_precision score of Logistic Regression model on training set is  [0.86261683 0.68377194 0.72323117 0.7429452  0.83697549 0.75682005
 0.79131151 0.6789852  0.71813048 0.76600983]
The recall score of Logistic Regression model on training set is  [0.60869565 0.4057971  0.4057971  0.47826087 0.57352941 0.42647059
 0.47058824 0.44117647 0.55882353 0.44927536]
The roc_auc score of Logistic Regression model on training set is  [0.98746515 0.9675988  0.97752666 0.9587031  0.98896519 0.97370865
 0.98055412 0.94456681 0.96977844 0.96513646]


In [57]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score, roc_auc_score

lr_tfidf_pre_accuracy = accuracy_score(lr_tfidf.predict(X_test_tfidf), y_test_tfidf)
lr_tfidf_pre_average_precision = average_precision_score(lr_tfidf.predict(X_test_tfidf), y_test_tfidf)
lr_tfidf_pre_recall = recall_score(lr_tfidf.predict(X_test_tfidf), y_test_tfidf)
lr_tfidf_pre_f1 = f1_score(lr_tfidf.predict(X_test_tfidf), y_test_tfidf)
lr_tfidf_pre_roc_auc = roc_auc_score(lr_tfidf.predict(X_test_tfidf), y_test_tfidf)

print("The accuracy score of Logistic Regression model on test set is", lr_tfidf_pre_accuracy)
print("The average precision score of Logistic Regression model on test set is", lr_tfidf_pre_average_precision)
print("The recall score of Logistic Regression model on test set is", lr_tfidf_pre_recall)
print("The f1 score of Logistic Regression model on test set is", lr_tfidf_pre_f1)
print("The roc_auc score of Logistic Regression model on test set is", lr_tfidf_pre_roc_auc)

The accuracy score of Logistic Regression model on test set is 0.9717561521252797
The average precision score of Logistic Regression model on test set is 0.45748443499101665
The recall score of Logistic Regression model on test set is 0.8636363636363636
The f1 score of Logistic Regression model on test set is 0.6529209621993127
The roc_auc score of Logistic Regression model on test set is 0.9194119498504958


## 3.2 Naive Bayes

In [46]:
from sklearn.naive_bayes import MultinomialNB

nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train_tfidf)

MultinomialNB()

In [51]:
from sklearn.model_selection import cross_val_score

measure_metrics = ['accuracy', 'f1', 'average_precision', 'recall', 'roc_auc']
for sco_name in measure_metrics:
    score = cross_val_score(nb_tfidf, X_train_tfidf, y_train_tfidf, scoring=sco_name, cv=10)
    print("The", sco_name, "score of Naive Bayes model on training set is ", score)

The accuracy score of Naive Bayes model on training set is  [0.9594689  0.94129979 0.95807128 0.94549266 0.94545455 0.95314685
 0.95594406 0.94195804 0.95104895 0.95104895]
The f1 score of Naive Bayes model on training set is  [0.5915493  0.4        0.55223881 0.42647059 0.44285714 0.464
 0.53333333 0.45751634 0.5        0.453125  ]
The average_precision score of Naive Bayes model on training set is  [0.69500647 0.47180853 0.61963895 0.44615176 0.5505796  0.53638567
 0.60198423 0.51700996 0.55298859 0.52623393]
The recall score of Naive Bayes model on training set is  [0.60869565 0.4057971  0.53623188 0.42028986 0.45588235 0.42647059
 0.52941176 0.51470588 0.51470588 0.42028986]
The roc_auc score of Naive Bayes model on training set is  [0.96108664 0.9252591  0.95103109 0.92036434 0.94831347 0.92778786
 0.94504189 0.92708603 0.93012007 0.92182858]


In [55]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score, roc_auc_score

nb_tfidf_pre_accuracy = accuracy_score(nb_tfidf.predict(X_test_tfidf), y_test_tfidf)
nb_tfidf_pre_average_precision = average_precision_score(nb_tfidf.predict(X_test_tfidf), y_test_tfidf)
nb_tfidf_pre_recall = recall_score(nb_tfidf.predict(X_test_tfidf), y_test_tfidf)
nb_tfidf_pre_f1 = f1_score(nb_tfidf.predict(X_test_tfidf), y_test_tfidf)
nb_tfidf_pre_roc_auc = roc_auc_score(nb_tfidf.predict(X_test_tfidf), y_test_tfidf)

print("The accuracy score of Naive Bayes model on test set is", nb_tfidf_pre_accuracy)
print("The average precision score of Naive Bayes model on test set is", nb_tfidf_pre_average_precision)
print("The recall score of Naive Bayes model on test set is", nb_tfidf_pre_recall)
print("The f1 score of Naive Bayes model on test set is", nb_tfidf_pre_f1)
print("The roc_auc score of Naive Bayes model on test set is", nb_tfidf_pre_roc_auc)

The accuracy score of Naive Bayes model on test set is 0.9460290827740492
The average precision score of Naive Bayes model on test set is 0.2525107979412443
The recall score of Naive Bayes model on test set is 0.46774193548387094
The f1 score of Naive Bayes model on test set is 0.4741144414168937
The roc_auc score of Naive Bayes model on test set is 0.7200066609572747


## 3.3 Random Forest

In [58]:
from sklearn.ensemble import RandomForestClassifier

rf_tfidf = RandomForestClassifier()
rf_tfidf.fit(X_train_tfidf, y_train_tfidf)

RandomForestClassifier()

In [59]:
from sklearn.model_selection import cross_val_score

measure_metrics = ['accuracy', 'f1', 'average_precision', 'recall', 'roc_auc']
for sco_name in measure_metrics:
    score = cross_val_score(rf_tfidf, X_train_tfidf, y_train_tfidf, scoring=sco_name, cv=10)
    print("The", sco_name, "score of Random Forest model on training set is ", score)

The accuracy score of Random Forest model on training set is  [0.97973445 0.98043326 0.97903564 0.97414396 0.98251748 0.97972028
 0.97412587 0.97692308 0.97902098 0.97692308]
The f1 score of Random Forest model on training set is  [0.77876106 0.75       0.68571429 0.65384615 0.76363636 0.72222222
 0.68571429 0.69230769 0.71559633 0.7037037 ]
The average_precision score of Random Forest model on training set is  [0.93241311 0.89103638 0.8958237  0.90489829 0.92941557 0.91869659
 0.90678407 0.8097739  0.88808589 0.8991531 ]
The recall score of Random Forest model on training set is  [0.57971014 0.60869565 0.57971014 0.49275362 0.63235294 0.61764706
 0.52941176 0.52941176 0.57352941 0.55072464]
The roc_auc score of Random Forest model on training set is  [0.99174807 0.98742259 0.99029028 0.99332291 0.99322471 0.99122722
 0.99306275 0.97118209 0.99264706 0.99092739]


In [60]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score, roc_auc_score

rf_tfidf_pre_accuracy = accuracy_score(rf_tfidf.predict(X_test_tfidf), y_test_tfidf)
rf_tfidf_pre_average_precision = average_precision_score(rf_tfidf.predict(X_test_tfidf), y_test_tfidf)
rf_tfidf_pre_recall = recall_score(rf_tfidf.predict(X_test_tfidf), y_test_tfidf)
rf_tfidf_pre_f1 = f1_score(rf_tfidf.predict(X_test_tfidf), y_test_tfidf)
rf_tfidf_pre_roc_auc = roc_auc_score(rf_tfidf.predict(X_test_tfidf), y_test_tfidf)

print("The accuracy score of Random Forest model on test set is", rf_tfidf_pre_accuracy)
print("The average precision score of Random Forest model on test set is", rf_tfidf_pre_average_precision)
print("The recall score of Random Forest model on test set is", rf_tfidf_pre_recall)
print("The f1 score of Random Forest model on test set is", rf_tfidf_pre_f1)
print("The roc_auc score of Random Forest model on test set is", rf_tfidf_pre_roc_auc)

The accuracy score of Random Forest model on test set is 0.979586129753915
The average precision score of Random Forest model on test set is 0.5970149509481706
The recall score of Random Forest model on test set is 0.990909090909091
The f1 score of Random Forest model on test set is 0.7491408934707904
The roc_auc score of Random Forest model on test set is 0.9850679326443897


## 3.4 Neural Network for Bert

In [ ]:
import tensorflow as tf

# Define the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_concat.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_bert_concat, y_train_bert, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test_concat)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test_bert, y_pred_binary)
print("Accuracy:", accuracy)